In [79]:
words = open('names.txt', 'r').read().splitlines()
words[:5]

['emma', 'olivia', 'ava', 'isabella', 'sophia']

In [80]:
block_size = 3 # given block_size character to predict the next
batch_size = 32

In [81]:
chars = list(set(''.join(words)))
chars = sorted(chars)
stoi = {c: i+1 for i, c in enumerate(chars)}
stoi['.'] = 0
iost = dict(zip(stoi.values(), stoi.keys()))

In [82]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
X, Y = [], []
output = []

for word in words:

    context = [0] * block_size
    for ch in word:
        idx = stoi[ch]
        context = context[1:] + [idx]
        X.append(context)
        Y.append(idx)

X = torch.tensor(X)
Y = torch.tensor(Y)

tr = int(len(X) * 0.8)
val = tr + int(len(X) * 0.1)
te = val + int(len(X) * 0.1)
X_tr = X[: tr]
Y_tr = Y[: tr]
X_val = X[tr: val]
Y_val = Y[tr: val]
X_te = X[val: te]
Y_te = Y[val: te]
print(f"{X_tr.shape=},{X_val.shape=},{X_te.shape=}\n"
      +f"{Y_tr.shape=},{Y_val.shape=},{Y_te.shape=}")

X_tr.shape=torch.Size([156890, 3]),X_val.shape=torch.Size([19611, 3]),X_te.shape=torch.Size([19611, 3])
Y_tr.shape=torch.Size([156890]),Y_val.shape=torch.Size([19611]),Y_te.shape=torch.Size([19611])


In [83]:
C = torch.randn(27, 2).requires_grad_(True)
emb = C[X_tr]
emb.shape

torch.Size([156890, 3, 2])

In [84]:
W1 = torch.randn(6, 100)
b1 = torch.rand(100)
W2 = torch.randn(100, 27)
b2 = torch.rand(27)
parameters = [C, W1, W2, b1, b2]

for p in parameters:
    p.requires_grad = True

In [85]:
sm = torch.nn.Softmax()

In [86]:
# key forward pass functions
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
logits = sm(h @ W2 + b2)

c:\Users\Administrator\anaconda3\lib\site-packages\torch\nn\modules\module.py:1511: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


In [95]:
loss = 0
for _ in range(50000):
    ix = torch.randint(0, X_tr.shape[0], (32,))
    emb = C[X_tr[ix]]
    h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
    logits = sm(h @ W2 + b2)
    loss = F.cross_entropy(logits, Y_tr[ix])
    for p in parameters:
        p.grad = None
    loss.backward()
    for p in parameters:
        p.data -= 0.1 * p.grad

print(loss.item())

2.756382942199707
